# DEIA Analytics Explorer (Plotly v2)

This notebook uses pandas + plotly (no matplotlib) to explore DEIA analytics data.
It also includes an environment check cell to verify which Python/kernel you are using.

## 0) Environment Check
Verifies interpreter, kernel, and package versions. If packages are missing, you can install into THIS kernel using the commented code.

In [ ]:
import sys, subprocess, site
print('Python version:', sys.version)
print('Executable   :', sys.executable)
print('User site    :', site.getusersitepackages())

def ensure(pkg):
    try:
        __import__(pkg)
        print(f'[ok] {pkg} already installed')
        return True
    except Exception:
        print(f'[miss] {pkg} not installed')
        return False

# To install into this exact kernel, uncomment lines below:
# if not ensure('pandas'):
#     subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'pandas'])
# if not ensure('plotly'):
#     subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'plotly'])


## 1) Imports & Paths
Locates the project root (folder containing `.deia`) and staging directory.

In [ ]:
import json
from pathlib import Path
try:
    import pandas as pd
    import plotly.express as px
    HAVE_PANDAS=True; HAVE_PLOTLY=True
except Exception as e:
    HAVE_PANDAS=False; HAVE_PLOTLY=False; print('[warn] pandas/plotly missing:', e)

def find_project_root(start: Path = Path.cwd()) -> Path:
    p = start.resolve()
    for _ in range(8):
        if (p/'.deia').is_dir(): return p
        if p.parent==p: break
        p = p.parent
    return start.resolve()
PROJECT_ROOT = find_project_root()
STAGING_DIR  = PROJECT_ROOT / '.deia' / 'analytics' / 'staging'
print('PROJECT_ROOT =', PROJECT_ROOT)
print('STAGING_DIR  =', STAGING_DIR)


## 2) Load Staging NDJSON Tables

In [ ]:
TABLES = ['sessions','session_decisions','session_action_items','session_files_modified','events','heartbeats']
def load_ndjson_table(table: str):
    files = sorted((STAGING_DIR / table).glob('dt=*/*.ndjson'))
    rows=[]
    for fp in files:
        for line in fp.read_text(encoding='utf-8', errors='replace').splitlines():
            s=line.strip();
            if not s: continue
            try: rows.append(json.loads(s))
            except Exception: pass
    return pd.DataFrame(rows) if HAVE_PANDAS else rows
DATA = {t: load_ndjson_table(t) for t in TABLES}
{t: (len(df) if HAVE_PANDAS else len(df)) for t, df in DATA.items()}


## 3) Utility — Pareto with Cumulative %
Given a Series of counts, render a Pareto chart (counts + cumulative %).

In [ ]:
def pareto_dataframe(series):
    df = series.sort_values(ascending=False).reset_index()
    df.columns=['category','count']
    total = df['count'].sum() or 1
    df['cum_count'] = df['count'].cumsum()
    df['cum_pct'] = (df['cum_count']/total*100).round(2)
    return df

def plot_pareto(df, title='Pareto', category='category', count_col='count'):
    if not HAVE_PLOTLY: return print('[info] plotly not available')
    fig = px.bar(df, x=category, y=count_col, title=title)
    fig2 = px.line(df, x=category, y='cum_pct')
    for d in fig2.data:
        d.yaxis = 'y2'
        d.name = 'Cum %'
    fig.update_layout(
        yaxis=dict(title='Count'),
        yaxis2=dict(title='Cumulative %', overlaying='y', side='right', range=[0,100]),
        legend=dict(orientation='h')
    )
    for d in fig2.data:
        fig.add_trace(d)
    fig.show()


## 4) Events — Pareto by event_type (with cumulative %)

In [ ]:
if HAVE_PANDAS and HAVE_PLOTLY and len(DATA['events'])>0:
    ev = DATA['events']
    if 'event_type' in ev.columns:
        dfp = pareto_dataframe(ev['event_type'].value_counts())
        plot_pareto(dfp, title='Events Pareto — event_type')
    else: print('[info] events has no event_type')
else: print('[info] events empty or plotly/pandas missing')


## 5) Events per Day (Event Time)
Counts by event timestamp. In Phase 2 we can also add ingestion date once we capture source paths.

In [ ]:
if HAVE_PANDAS and HAVE_PLOTLY and len(DATA['events'])>0:
    ev = DATA['events'].copy()
    ts_col = 'ts' if 'ts' in ev.columns else ('timestamp' if 'timestamp' in ev.columns else None)
    if ts_col:
        ev['_ts'] = pd.to_datetime(ev[ts_col], errors='coerce', utc=True)
        by_event = ev.dropna(subset=['_ts']).set_index('_ts').resample('D').size().to_frame('by_event').reset_index()
    else:
        by_event = pd.DataFrame(columns=['_ts','by_event'])
    # ingestion date approximation from partition folder in path if present
    if 'source_path' in ev.columns:
        # not available in Phase 1; future enhancement
        pass
    by_event.rename(columns={'_ts':'date'}, inplace=True)
    fig = px.line(by_event, x='date', y='by_event', title='Events per Day (by event timestamp)')
    fig.show()
else: print('[info] events empty or plotly/pandas missing')


## 6) Sessions — Start vs. Ingested Dates
Plots sessions per day by `ts_start` and also lists `ts_ingested` to confirm recency.

In [ ]:
if HAVE_PANDAS and HAVE_PLOTLY and len(DATA['sessions'])>0:
    ss = DATA['sessions'].copy()
    ss['_start'] = pd.to_datetime(ss.get('ts_start', None), errors='coerce', utc=True)
    s_by_start = ss.dropna(subset=['_start']).set_index('_start').resample('D').size().to_frame('sessions_by_start').reset_index()
    fig = px.line(s_by_start, x='_start', y='sessions_by_start', title='Sessions per Day (by ts_start)')
    fig.show()
    if 'ts_ingested' in ss.columns:
        ss['_ing'] = pd.to_datetime(ss['ts_ingested'], errors='coerce', utc=True)
        print('Most recent ts_ingested:', ss['_ing'].max())
else: print('[info] sessions empty or plotly/pandas missing')


## 7) Heartbeats — Last Seen per Bot (Bar)
Shows most recent heartbeat times per bot.

In [ ]:
if HAVE_PANDAS and HAVE_PLOTLY and len(DATA['heartbeats'])>0:
    hb = DATA['heartbeats'].copy()
    ts_col = 'ts' if 'ts' in hb.columns else ('timestamp' if 'timestamp' in hb.columns else None)
    if ts_col and 'bot_id' in hb.columns:
        hb['_ts'] = pd.to_datetime(hb[ts_col], errors='coerce', utc=True)
        hb2 = hb.dropna(subset=['_ts']).sort_values('_ts').groupby('bot_id').tail(1)[['bot_id','_ts']].drop_duplicates('bot_id').sort_values('_ts')
        fig = px.bar(hb2, x='_ts', y='bot_id', orientation='h', title='Heartbeats — Last Seen per Bot')
        fig.show()
    else: print('[info] missing bot_id or timestamp in heartbeats')
else: print('[info] heartbeats empty or plotly/pandas missing')


## 8) Tasks → Responses Lead Time (Heuristic)
Matches TASK and RESPONSE by subject (from filenames) and plots lead time distribution (minutes).

In [ ]:
if HAVE_PANDAS and HAVE_PLOTLY and 'hive_tasks' in DATA and 'hive_responses' in DATA:
    tk = DATA['hive_tasks'].copy(); rp = DATA['hive_responses'].copy()
    if len(tk)>0 and len(rp)>0 and 'subject' in tk.columns and 'subject' in rp.columns:
        tk['_ts'] = pd.to_datetime(tk['ts'], errors='coerce', utc=True)
        rp['_ts'] = pd.to_datetime(rp['ts'], errors='coerce', utc=True)
        # For each task subject, find earliest response with same subject
        first_resp = rp.sort_values('_ts').groupby('subject').head(1)[['subject','_ts']].rename(columns={'_ts':'resp_ts'})
        merged = tk[['subject','_ts']].rename(columns={'_ts':'task_ts'}).merge(first_resp, on='subject', how='left')
        merged = merged.dropna(subset=['task_ts','resp_ts'])
        merged['lead_minutes'] = (merged['resp_ts'] - merged['task_ts']).dt.total_seconds()/60.0
        merged = merged[merged['lead_minutes']>=0]  # keep non-negative
        if len(merged)>0:
            fig = px.histogram(merged, x='lead_minutes', nbins=30, title='TASK→RESPONSE Lead Time (minutes)')
            fig.show()
            # Pareto of subjects by count of tasks
            dfp = pareto_dataframe(tk['subject'].value_counts())
            plot_pareto(dfp, title='Tasks Pareto — subject')
        else: print('[info] no matched TASK→RESPONSE pairs')
    else:
        print('[info] missing subject columns or empty tables')
else:
    print('[info] hive tasks/responses not available')


## 9) Ideas for AI Engineer & COO Dashboards
- AI Engineer:
  - Pareto of events by type, by agent, by subject (with cumulative %)
  - Error/alert rate over time; TASK→RESPONSE lead time trend
  - Heartbeat gap analysis (downtime per agent)
  - Decisions/action items per session; top files modified
- COO:
  - Throughput (sessions/day), WIP (open tasks), lead time (assign→complete), on‑time completion rate
  - Blockers/incidents over time; SLA (mean/95th) for responses
  - Utilization: active hours per agent (from heartbeats/events)
  - Season/Flight KPIs (when docs align)